In [1]:
load_model = 'densenet_based_model2.pt'

data_dir = '../../../stanford_dogs_new/'

In [2]:
# import numpy as np

from IPython.display import Image

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torchvision.datasets import ImageFolder
from PIL import Image

from torch.autograd import Variable

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


from IPython.display import Image
from IPython.display import Markdown

from PIL import Image
import PIL.Image


## load the Densenet121 based model

In [3]:
densenet121_loaded = models.densenet121(pretrained=True)

for param in densenet121_loaded.parameters():
    param.requires_grad = False

    
densenet121_loaded.classifier = nn.Sequential(*list(densenet121_loaded.classifier.children())[:-1] + [nn.Linear(in_features=1024, out_features=120, bias=True)])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
densenet121_loaded = densenet121_loaded.to(device)



densenet121_loaded.load_state_dict(torch.load('densenet121_based_model2.pt'))
densenet121_loaded.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [4]:
model = densenet121_loaded

In [5]:
# preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
])

In [6]:
path = '../../../stanford_dogs_new/test'

list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

# gets the breed name from subfolders name like 'n02099429-curly-coated_retriever'
def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

per_class_accuracy = dict()
total_accuracy = 0

for breed_dir in list_subfolders_with_paths:
    
    breed = rename(breed_dir.split('/')[-1])
    
    images = os.listdir(breed_dir)
    
    matched = False
    matches = 0
    total_images = len(images)

    for image in images:

        input_image = Image.open(breed_dir + '/' + image)


        input_tensor = preprocess(input_image)
        if torch.cuda.is_available():
            input_tensor = Variable(input_tensor.cuda())

        input_batch = input_tensor.unsqueeze(0)
        out = model(input_batch)

        probabilities = torch.nn.functional.softmax(out[0], dim=0)
        # print(probabilities)

        with open("../../stanford_dogs_breeds_classes_final.txt", "r") as f:
            categories = [s.strip() for s in f.readlines()]

        predicted_breeds = []
        top1_prob, top1_catid = torch.topk(probabilities, 1)
        for i in range(top1_prob.size(0)):
            # predicted_breeds.append([categories[top3_catid[i]], top3_prob[i].item()*100])
            predicted_breed = categories[top1_catid[i]]

        # list to be used directly by the application (predicted_breed, probability)
        # print("predicted_breeds are: \n", predicted_breed)
        if breed == predicted_breed:
            matched = True
            matches = matches + 1

        per_class_accuracy[breed] = matches / total_images * 100


In [7]:
per_class_accuracy

{'Pomeranian': 95.45454545454545,
 'basset': 88.88888888888889,
 'Japanese spaniel': 100.0,
 'Bouvier des Flandres': 73.33333333333333,
 'German shepherd': 75.0,
 'Brabancon griffon': 81.25,
 'Blenheim spaniel': 94.73684210526315,
 'kelpie': 75.0,
 'Shih Tzu': 95.45454545454545,
 'curly coated retriever': 87.5,
 'Lakeland terrier': 50.0,
 'Irish setter': 100.0,
 'German short haired pointer': 100.0,
 'flat coated retriever': 87.5,
 'Irish water spaniel': 93.33333333333333,
 'Dandie Dinmont': 100.0,
 'clumber': 100.0,
 'Yorkshire terrier': 47.05882352941176,
 'schipperke': 93.75,
 'briard': 75.0,
 'miniature poodle': 50.0,
 'chow': 100.0,
 'Saluki': 95.0,
 'boxer': 56.25,
 'Pekinese': 66.66666666666666,
 'komondor': 93.75,
 'Brittany spaniel': 75.0,
 'black and tan coonhound': 81.25,
 'Appenzeller': 37.5,
 'Eskimo dog': 13.333333333333334,
 'giant schnauzer': 68.75,
 'Old English sheepdog': 100.0,
 'Chihuahua': 81.25,
 'miniature schnauzer': 100.0,
 'redbone': 73.33333333333333,
 'wire 

In [8]:
total_accuracy = sum(per_class_accuracy.values())/120
total_accuracy

81.99398446196048

In [9]:
sorted(per_class_accuracy.items(), key=lambda x: x[1])

[('Eskimo dog', 13.333333333333334),
 ('Appenzeller', 37.5),
 ('Walker hound', 37.5),
 ('collie', 37.5),
 ('standard schnauzer', 43.75),
 ('Yorkshire terrier', 47.05882352941176),
 ('Lhasa', 47.368421052631575),
 ('Lakeland terrier', 50.0),
 ('miniature poodle', 50.0),
 ('boxer', 56.25),
 ('standard poodle', 56.25),
 ('Border collie', 60.0),
 ('soft coated wheaten terrier', 62.5),
 ('Great Pyrenees', 63.63636363636363),
 ('American Staffordshire terrier', 64.70588235294117),
 ('Australian terrier', 65.0),
 ('Pekinese', 66.66666666666666),
 ('toy terrier', 66.66666666666666),
 ('whippet', 68.42105263157895),
 ('giant schnauzer', 68.75),
 ('Cardigan', 68.75),
 ('English foxhound', 68.75),
 ('toy poodle', 68.75),
 ('Siberian husky', 70.0),
 ('Greater Swiss Mountain dog', 70.58823529411765),
 ('Labrador retriever', 72.22222222222221),
 ('Bouvier des Flandres', 73.33333333333333),
 ('redbone', 73.33333333333333),
 ('kuvasz', 73.33333333333333),
 ('German shepherd', 75.0),
 ('kelpie', 75.0),

### Top 10 Breeds with least accuracy

In [10]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1])[:k]

[('Eskimo dog', 13.333333333333334),
 ('Appenzeller', 37.5),
 ('Walker hound', 37.5),
 ('collie', 37.5),
 ('standard schnauzer', 43.75),
 ('Yorkshire terrier', 47.05882352941176),
 ('Lhasa', 47.368421052631575),
 ('Lakeland terrier', 50.0),
 ('miniature poodle', 50.0),
 ('boxer', 56.25)]

### Top 10 Breeds with most accuracy

In [11]:
k = 15
sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True)[:k]

[('Japanese spaniel', 100.0),
 ('Irish setter', 100.0),
 ('German short haired pointer', 100.0),
 ('Dandie Dinmont', 100.0),
 ('clumber', 100.0),
 ('chow', 100.0),
 ('Old English sheepdog', 100.0),
 ('miniature schnauzer', 100.0),
 ('keeshond', 100.0),
 ('Samoyed', 100.0),
 ('bull mastiff', 100.0),
 ('Rottweiler', 100.0),
 ('Doberman', 100.0),
 ('Bernese mountain dog', 100.0),
 ('Kerry blue terrier', 100.0)]